# Notes on Spectral Line Models

`simple_disk.py` is the code that Sean Andrews and Rich Teague created to model spectral line emissions from disk

---
## Disk Basics

from [here](https://arxiv.org/pdf/2001.05007.pdf)
* Disks form after the core of a molecular cloud has condensed and accreted (gravitationally collapsed inwards)
    * Right after star has formed (pre-main sequence)
    * Small rotational nudge to the core is equaled out via conservation of angular momentum of the outer cloud, forming disks
    
* Analyzing the spectral line emission of a disk is (of course) useful to know the properties of the gas in disk
* isotope is to element as isotopologue is to molecule
    * HD (tracer element) is isotopologue to $H^2$

* __Inclination:__ The tilt of the disk towards our line of sight (degrees)
    * `inc = 0`: face on
    * `inc = 90`: edge on
* __Position Angle:__ `PA` Angle (in degrees) between the red-shifted major axis and North, measured from North to East (up to left, disk frame)

### Coordinate Systems
* __Sky-frame:__ in arcseconds, view of disk in the celestial sky (imagine looking up at it from Earth)
    * Coordinates: 
        * Cartesian: `disk.x_sky` `disk.y_sky`
        * Polar: `disk.r_sky` `disk.t_sky`
    * Axes: `disk.xaxis_sky` `disk.yaxis_sky` (need to flip x if plotting)
* __Disk frame:__ in AU (1 AU = 1.496e13 cm)
    * Coordinates: 
        * Cartesian: `disk.x_disk` `disk.y_disk`
        * Polar: `disk.r_disk` `disk.t_disk` (disk polar angles are not the same as PA)
    * Axes: `disk.xaxis_disk` `disk.yaxis_disk`
* Conversion: Offsets in arcseconds correspond to offsets in AU divided by the distance in parsecs
    * 1 pc = $3.09 \times 10^{18}$ cm = 3.26 light years
    
### Disk Properties
* Background keplerian rotation of disk (disk frame):
$$v_{\phi}(r) = \sqrt{\frac{GM_{\rm star}}{r}}$$
* Projected onto the sky (sky frame)
$$v_{\phi,\, {\rm proj}}(r,\, \phi) = \sqrt{\frac{GM_{\rm star}}{r}} \times \sin(i) \times \cos (\phi)$$

#### Emission
* Model assumes that the distribution of spectral emission of a disk is made up of gaussian profiles:
    * Gaussian Profile is gaussian function (function that produces gaussian bell curve) of the form:

    $$P(x) = a \ \exp \frac{-(x-\bar{x})^2}{2\sigma ^2}$$

    where $a$ is the height of the curve's peak (peak value), $\bar{x}$ is the mean, and $\sigma$ is the standard deviation. The expression in the denominator of the exponential is referred to as the FWHM linewidth

* __In terms of emission:__
    <blockquote>
    $$
    I_{\nu} = T_{\rm B} \times \exp \left[ -\left( \frac{v - v_0}{\Delta v} \right)^2 \right]. 
    $$

    (Remember that $I_{\nu}$ is spectral radiance [mJy]!) The line center, $v_0$, is governed by the projected velocity field.  The emission is assumed to come from some vertical height above the disk midplane, `z_l`.  We'll get back to that later: first, we'll assume `z_l = 0` (this is the default).  
    </blockquote>

    * $T_{b}$, the __peak temperature value__, is defined by the radial power law:

    $$
    T_{\rm B} = T_{{\rm B},\,0} \times \left( \frac{r}{r_0} \right)^{-q_{T_{\rm B}}}
    $$

    where the appropriate parameters are `Tb0` and `Tbq`.  Note that `r0` is the radius where `Tb0` is defined; it defaults to 10 AU.  The outer boundary of the emission distribution is set by the parameter `r_l`: it defaults to 100 AU.  We can also define maximum and minimum temperature boundaries, with `Tbmax` and `Tbmin`.
    
    * The __(doppler) linewidth__, $\Delta v$, has several default parameterizations:  
        * Simple case (radial power-law)
        <blockquote>
        $$\Delta v(r) = \Delta v_0 \times \left( \frac{r}{r_0} \right)^{q_{\Delta v}}$$

        where the appropriate parameters are `dV0`, `dVq`.
        </blockquote>
        * More complex case:
        <blockquote>
        The linewidth is (thought to be) associated with temperature structure: the gas has some thermal contribution to the linewidth and a non-thermal one.  These are both thought to be proportional to the local sound speed, $c_s$,

        $$c_s(r) = \left( \frac{k T}{\mu_{\rm g} m_{\rm H}} \right)^{0.5}$$

        where $k$ is Boltzmann's constant, $\mu_{\rm g}$ is the mean molecular weight of the gas (= 2.37, which is an appropriate mix of H, He, and molecules), and $m_{\rm H}$ is the mass of a H atom.  In this preferred scenario, we associate $T$ with $T_B$ and let:

        $$\Delta v(r) = \left[\frac{2 k T_b}{\mu_\ell m_{\rm H}} + (\xi \,  c_s)^2\right]^{0.5} = 
        c_s \left[2 \frac{\mu_{\rm g}}{\mu_\ell} + \xi^2\right]^{0.5}$$

        where $\mu_\ell$ is the mean molecular weight of the molecule that's producing the spectral line (this defaults to CO, where $\mu_\ell = 28$) and $\xi$ is the fraction of the sound speed of the gas that's attributable to non-thermal motions (this is usually associated with turbulence).  So, the relevant parameter here is $\xi$, or `xi_nt`, which can vary from 0 to 1 (and is usually near 0).
        </blockquote>
        * Is not related to FWHM of typical gaussian profile
            * But is FWHM of the intensity of the spectral emission
        
### Altering Parameters
__Code to use to change things:__ 

<code>disk = simple_disk(inc=30, PA=45., mstar = 1, FOV=8.0, r_l=300, xi_nt = 0.05, dVmax=500, Tb0=200.0, Tbq=0.5, Tbmax=500)</code>

From `simple_disk.py`:
<blockquote>

* __Geometric Parameters__:
    * inc (float): Inclination of the source in [degrees].
    * PA (float): Position angle of the source in [degrees].
    * x0 (Optional[float]): Source center offset along x-axis in [arcsec].
    * y0 (Optional[float]): Source center offset along y-axis in [arcsec].
    * dist (Optional[float]): Distance to the source in [pc].
    * mstar (Optional[float]): Mass of the central star in [Msun].
    * r_min (Optional[float]): Inner radius in [au].
    * r_max (Optional[float]): Outer radius in [au].
    * r0 (Optional[float]): Normalization radius in [au]. (r0 must be < r_l)
    * r_l (Optional[float]): Turn-over radius in [au].
    * z0 (Optional[float]): Emission height in [au] at r0.
    * zpsi (Optional[float]): Index of z_l profile for r < r_l.
    * zphi (Optional[float]): Exponential taper index of z_l profile at r > r_l.

* __Brightness Temperatures:__
    * Tb0 (Optional[float]): Brightness temperature in [K] at r0.
    * Tbq (Optional[float]): Index of Tb profile for r < r_l.
    * Tbeps (Optional[float]): Exponential taper index of Tb profile for r > r_l.
    * Tbmax (Optional[float]): Maximum Tb in [K].
    * Tbmax_b (Optional[float]): Maximum Tb for back side of disk in [K].

* __Optical depth of front-side:__
    * tau0 (Optional[float]): Optical depth at r0.
    * tauq (Optional[float]): Index of optical depth profile for r < r_l
    * taueta (Optional[float]): Exponential taper index for optical depth profile at r > r_l.
    * taumax (Optional[float]): Maximum optical depth.

* __Line-widths:__
    * dV0 (Optional[float]): Doppler line width in [m/s] at r0.
    * dVq (Optional[float]): Index of line-width profile.
    * dVmax (Optional[float]): Maximum line-width.
    * xi_nt (Optional[float]): Non-thermal line-width fraction (of sound speed for the gas); can use if dV0, dVq are None.

* __Observational Parameters:__
    * FOV (Optional[float]): Field of view of the model in [arcsec].
    * Npix (Optional[int]): Number of pixels along each axis.
    * mu_l (Optional[float]): Mean atomic weight for line of interest.
</blockquote>

#### Trends
* `mstar` $\uparrow$ — temp $\uparrow$
* `r0` $\uparrow$ — temp $\uparrow$ (no change in size)
* `r_l` $\uparrow$ — no change in temp, only size $\uparrow$
* `xi_nt` $\uparrow$ – slight change in amount of spectrum
* `Tb0` $\uparrow$ – change in hottest temperatures of spectra
* `Tbq` $\downarrow$ – temp $\uparrow$ (only changes hottest temperatures of spectra)

#### 9/10 Notes
* On __doppler width__:
    * Intrinsically related to the spectrum, little relation to channel map velocities/systemic velocities
    * The complex doppler formula is matter of preference (varies within astrophysics community)
        * If you want to measure turbulence/its effects, use the more complex one
        * Simpler one is more practical, has more clear relationship with radius
    
* The `bmaj` argument in the channel maps controls the intrinsic spatial imperfections of the instrument
    * Factors like atmosphere, % can blur the image
    
* `r0`, `r_l` is the min/max related to the power law
* `r_min` and `r_max` are quasi-archaic, related to variables like `Tbeps`, but generally define the boundaries of the disk in the grid

* Polar angle is what does the spinning/rotating
    * Define polar angle based on the red-shift of the major axis
        * if PA = 0, then major axis will show North to South
        * if PA = 90, then major axis will be East to West
        
* Astrophysics community assumes that disks are azimuthally symmetric (symmetry around polar angle; rotation)
    * False
        
* simple_disk.py model doesn't account for specific transitions within individual atoms
    * identify quantum-mechanically (SAHA equation?) what the elements are doing, identify transitions and how many photons are being emitted by each molecule
    
    
* Current side project: developing radio transfer code that accounts for these smaller/more specific parameters in QM
    * Preliminary testing of model works well, but it messes up on the doppler linewidths
    * Predicts masses well though, mass affects structure of emission
    * tricky part is degeneracies:
        * mass and inclination are intertwined (linearly-dependent transformations?), they can accomplish the same things and it's hard to tell which one is doing what
        
        
* optical depth - how far you can see into the disk
    * changes with position
    * Large optical depth = optically thick -- light is readily absorbed (large probing region).
        * If gas is __optically thick__, then a photon will interact __more__ with particles before it finally escapes from the cloud.

## Project Timeline
---
* **Week 10:**
    * Goal: Learning `simple_disk.py` and playing with model
    